## Hyper-Parameter Tuning (Embedding model)
This document tunes hyperparameters for use in the "Category embedding model" that performs text classification of consumer complaints (using NLP-LSTM mode; see script [here](https://github.com/faraway1nspace/NLP_topic_embeddings/blob/master/FinComplain_LSTM_embedding_model.ipynb)), which is an elaboration of a more generic [LSTM-NLP model](https://github.com/faraway1nspace/NLP_topic_embeddings/blob/master/FinComplain_LSTM_default_model.ipynb); the category embedding is inspired by the famous [instacart model](https://tech.instacart.com/deep-learning-with-emojis-not-math-660ba1ad6cdc). The key hyperparameter is the embedding dimension of the category embeddings.

<b>Hyperparameters</b> should be done by minimizing a cross-validation loss. In this script, I use a <b>Thompson-sampler</b> (inspired by a cruide _reinforcement-learning_ / multi-arm bandit algorithm), to stochastically explore the high-dimensional hyperparameter space and find good hyperparameter values. This is an original algorithm and is untested, but it seems to work well.

The hyperparameters include:
+ the embedding dimension of the category embedding
+ the embedding dimension of the word-embedding (like word2vec) 
+ the size of the corpus used in the word-tokenization (which feeds into the word-embedding). 
+ the dimensionality of the Long-Short-Term-memory outputs
+ the Dropout rate in the LSTM

... as well as choosing total number of epochs to run the LSTM.

### Category Embeddings
The category embedding learns a low-dimensional representation of the >400 different financial complaint Issues in the US Financial Consumer Protection [complaint database](https://www.consumerfinance.gov/data-research/consumer-complaints/). The number of embedding dimensions should be much less than the number of categories, at most 1/2, and probably much less than the sqrt(#[categories]). More dimensions means a potentially better representation but higher variance and redundancy; fewer dimensions means more bias but lower variance: it is a classic trade-off, and should be treated as a hyperparameter. If the embeddings have no affect on predictive performance, then this should be reflected in uncertainty in the hyper-parameter tuning algorithm.

During an Insight Data fellowship, I used this technique on a similar proprietary dataset for sentiment analysis, and found it help us make sense of the (growing) number of consumer-product categories.

### Function overview
+ `run_model` The main function which runs an individual cross-validation run 
+ `proposal_hyperparam` The main thompson sampler function which proposes new samples from the hyperparameter space.

The procedure is straight forward
+ get <b>samples</b> from the hyperparameter space
+ do <b>3-fold cross-validation</b> to estimate an Expected Loss (aka hold-out loss)
+ use the Expected Loss as the 'reward' in a <b>multi-arm bandit learner</b>
+ calculate <b>probabilities</b> for each combination of hyperparameters
+ use the <b>Thompson-sampler</b>/multi-arm bandit algorithm to draw a new sample from the hyperparameter space
+ <b>repeat</b> for about 30 iterations.

The multi-arm bandit learner should progressively sample from the hyperparameter space that has a higher-probability of minimizing the Expected Loss.


## Functions: Data Import & Natural Language Pre-Processing

The following functions are some idiosyncratic functions to import and clean the Financial complaint data. For the background of the data source and the models' purpose, please see the [Readme file](https://github.com/faraway1nspace/NLP_topic_embeddings) as well as the [US Consumer Complaint Database](https://www.consumerfinance.gov/data-research/consumer-complaints/) at the US Consumer Financial Protection Bureau website.
+ `import_and_clean_data` : reads the complaint data and organizes it for the `keras` LSTM model
+ `nlp_preprocess` " does some basic NLP pre-processing (stemming, removing stop words, etc.)

In [1]:
# %matplotlib notebook
import os
import time
import pandas as pd
import numpy as np
import re
from math import log,exp
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import download as nltk_downloader 
from keras import backend as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, RepeatVector, concatenate, Dense, Reshape, Flatten
from keras.models import Model
from scipy.stats import rankdata as rd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import KFold # import KFold
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor

# set the working directory
#os.chdir("...")

# NLP function to replace english contractions
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# function to do some basic NLP pre-processing steps: replacing contractions, stemming words, removing stop words
def nlp_preprocess(text_column, # column in Panda table with text
                   stop_words, # list of English stopwords
                   word_clip = 300): # truncate the number of words
   # remove contractions
   ps = PorterStemmer()  # stemmer    
   cTextl = [decontracted(x) for x in text_column.values.tolist()]
   # remove double spacing and non-alphanumeric characters
   cTextl=[re.sub(' +',' ',re.sub(r'\W+', ' ', x)) for x in cTextl]
   # lower case the words
   cTextl = [x.lower() for x in cTextl]
   # stop words and stemming
   for i in range(0,len(cTextl)):
      rawtext = cTextl[i].split(" ") # splits sentence by spaces
      rawtext = rawtext[0:min(word_clip,len(rawtext))] # take only 300 words maximum
      # stem and remove stopwords in one line (expensive operation)
      newtext = " ".join(ps.stem(word) for word in rawtext if not word in stop_words)  # loop through words, stem,join
      cTextl[i] = newtext
   return pd.DataFrame(cTextl)

# function: import and pre-process the data (prepare for Keras)
def import_and_clean_data(filename, # file name of data to import (either a .csv or a tar.xz file of a .csv)
                          col_label = "Label3",
                          data_dir = "data/", # directory
                          tmp_dir = "/tmp/", # if file is a .tar.xz, where to temporarily extract the data (Windows users need specify differently than /tmp/
                          rare_categories_cutoff = 10, # threshold for categories to be included in the training set
                          word_clip = 300): # max number of words in text to accept (only first 300 words are retained
   # check
   if "tar.xz" in filename:
      print("decompressing " + filename + " into "+tmp_dir)
      # command for shell
      os_system_command = "tar xf "+data_dir+filename+" -C "+tmp_dir
      print(os_system_command)
      # run decompression command (for Linux/Mac)
      os.system(os_system_command)
      newfilename = tmp_dir + filename.split(".tar.xz")[0]
   else:
      print("importing csv called " + filename)
      newfilename = data_dir + filename
   # read the complaint data 
   d_raw = pd.read_csv(newfilename, usecols = ['State','Complaint ID','Consumer complaint narrative','Product', 'Sub-product', 'Issue', 'Sub-issue'])
   print("imported " + str(d_raw.shape[0]) + " rows of data") # notice 191829 rows and 7 columns
   # fill NaN with blanks
   for col_ in ['Product','Sub-product','Issue']:
      d_raw[col_] = d_raw[col_].fillna(" ") # fill NaN with a character
   # factorize the two levels (Product and Product+Issue) to get unique values
   d_raw['Label1'] = pd.factorize(d_raw['Product'])[0]
   # combine Product + Issues
   d_raw['Label3'] = pd.factorize(d_raw['Product'] + d_raw['Sub-product']+d_raw['Issue'])[0] # 570 Categories
   # Dictionary: category integers vs. category names
   cats = [pd.factorize(d_raw['Product'])[1],  pd.factorize(d_raw['Product'] + d_raw['Sub-product'])[1], pd.factorize(d_raw['Product'] + d_raw['Sub-product']+d_raw['Issue'])[1]]
   # truncate the data: only use categories with at least 10 observations
   labels_counts = d_raw.groupby([col_label]).size() # counts of Level3 categories 
   which_labels = np.where(labels_counts>=rare_categories_cutoff)[0] # which categories have at least 'cutoff'
   # make new (truncated) dataset
   ixSubset = d_raw.Label3.isin(which_labels) # subset integers
   # new dataset 'd', as subset of d_raw
   d = (d_raw[ixSubset]).copy()
   # NLP pre-processing: stopwords removal, stemming, etc.
   # get the default English stopwords from nlkt pacakge
   from nltk.corpus import stopwords 
   stop_words = set(stopwords.words('english')) # list of stopwords to remove
   # get the stemming object from nltk
   ps = PorterStemmer()  # stemmer
   # column in data with the Text data (to feed into the LSTM)
   col_text = 'Consumer complaint narrative' # name of the column with the text 
   # NLP: pre-process the text/complaints
   print("performing NLP pre-processing on column " + col_text + " (remove stop words, stemming,...). This may take a while...")
   cText = nlp_preprocess(d[col_text],stop_words, word_clip = word_clip)
   print("Done NLP pre-processing")
   # process the labels, make into a N-hot-coding matrix 
   Y = pd.get_dummies(d['Label3'].values) # one-hot coding
   # get integers representing each (label3) class (these are the column names)
   Ynames_int = Y.columns # notice the confusing mapping of different integers to different integers
   # get english issue labels corresponding to each integer value in Ynames_int
   Ynames_char = [cats[2][i] for i in Ynames_int] # actual names
   # Finally, convert Y into a numpy matrix (not a panda df)
   Y = Y.values
   n_obs = Y.shape[0]
   # make the input data for the category embedding
   uLabels3 = d['Label3'].unique() # unique level 3 labels
   nLabels3 = len(uLabels3) # number of unique level 3 labels
   X_labels  = np.repeat(np.array([i for i in range(0,nLabels3)],dtype=int).reshape(1,nLabels3),n_obs,axis=0) # input for embedding layer
   print("returning cleaned text data 'cText' for use in tokenization; 'Y' as an one-hot-coding matrix of categories; and 'cats' a dictionary matches columns in Y to english category names")
   if "tar.xz" in filename:
      print("deleting temporary file " + filename)
      os_system_command = "rm "+ newfilename
      print(os_system_command)
      os.system(os_system_command)
      print("done pre-processing")
   return cText, Y, X_labels,cats

# function to calculate sample_weights for keras argument sample_weight
def get_class_weights(Y, # N-hot-coding response matrix
                      clip_ = 100000): # maximum weight for rarer cases
   weights_total_class_counts = (Y).sum(axis=0)
   weights_by_class = (min(weights_total_class_counts)/weights_total_class_counts) # weight by the rarest case
   Y_int = np.argmax(Y,axis=1)
   vWeights_raw = np.array([weights_by_class[i] for i in Y_int], dtype=float)
   vWeights = np.clip(vWeights_raw * (Y.shape[0]/sum(vWeights_raw)),0,clip_)
   return vWeights


Using TensorFlow backend.


## Functions: Hyperparameter Sampling

With 4 hyperparameters and (at least) 3 discrete values each, I explored 81 possible _combinations_ of hyperparameter (for an academic or production-level project, you'd likely want more fine-grained hyperparameter values). The goal is to find the best combination of hyperparameter values, based on a cross-validation loss (Expected Loss). It is very inefficient to do cross-validation on EVERY combination. Instead, we'll do a <b>stochastic search</b> across the space of hyperparameter combinations, arriving at the best combination much earlier than running all 81 combinations.

#### Thompson sampling:
Stochastic exploration of the hyperparameter space requires some way to calculate _probabilities_ for each combination of hyperparameters. I use a simple principle from reinforcement learning called <b>Thompson sampling</b>: the more _uncertain_ a potential action/reward is, the more _likely_ we should try it, and thus reduce our uncertainty quickly find the most rewarding action. Here _action_ means picking a combination of hyperparameters to estimate the Expected Loss; and _reward_ is finding the lowest Expected Loss.

#### Model uncertainty as a Dirichlet-Multinomial process
The key innovation I present is my method to estimate the probabilities for each hyperparameter combination: this will involve an ensemble of penalized-regressors (`Ridge` and `DecisionTreeRegressor` from `sklearn`). The regressors will estimate the Expected Loss of each combination using the hyperparameter variables as predictor variables. The _frequency counts_ of each hyperparameter-combination being the best model (lowest predicted loss) is converted into a probability via the <b>Dirichlet-Multinomial</b> process. 

There are a bunch of functions, the main ones pertaining to the hyperparameters are:
 + `make_hyperparameters_combos` : takes a dictionary of hyperparameters values and makes a grid of all possible combinations
 + `optimal_order_of_hyperparameter_runs` : proposes an optimal sequence in which to run the different combination of hyperparameter values. This will find high-contrasting parameter-combinations, so that the space of hyperparameter values is quickly explored deterministically, prior to evoking the multi-arm bandit algorithm
 + `proposal_hyperparam`: main Thompson sampler; proposes new combinations of hyperparameters to run in cross-validation. It switches between a deterministic algorithm and a stochastic algorithm after a preset number of iterations (`toggle_learner`). There are other hyper-hyper-parameters that govern the learning behaviour of the multi-arm-bandit process (`ridge_alpha`, `max_depth`,`multinomial_prior`) and should be left as is.

In [2]:
# internal function for cross-validation: creates all combinations of hyperparameters
def make_hyperparameters_combos(hyper_parameters):
   # hyperparmeters' values
   hyp_args = [x[1] for x in hyper_parameters.items()]
   # names of hyperparameters
   hyp_names = [x[0] for x in hyper_parameters.items()]
   # all combos of hyperparameters, as a tensor
   hyp_args_grid = np.meshgrid(*hyp_args)
   # dimensions of the hyperparameters
   hyp_args_dimensions = list(hyp_args_grid[0].shape)
   # total number of hyperparameter combos
   total_combos = round(exp(sum(map(log,hyp_args_dimensions))))
   # reshape into a [n_combos,parameters]
   hyp_grid = (np.array(hyp_args_grid).reshape(len(hyp_args_grid),total_combos)).T
   # convert grid to data.frame
   hyp_pd = pd.DataFrame(hyp_grid, columns = hyp_names)
   # also make a companion sequence
   hyp_seq = [[j for j in range(0,len(x[1]))] for x in hyper_parameters.items()]
   hyp_seq_grid = (np.array(np.meshgrid(*hyp_seq)).reshape(len(hyp_args_grid),total_combos)).T
   # make an empty panda data.frame to fill with results
   empty_res = pd.DataFrame({"cv_loss": [0 for i in range(0,hyp_pd.shape[0])], "best_epoch": [0 for i in range(0,hyp_pd.shape[0])]})
   return hyp_pd, hyp_seq_grid, empty_res

# internal function for cross-validation: optimal order of running different hyperparameter scenarios: calculates a 'scenario distance' by finding scenarios that are maximally contrasting with each other
def optimal_order_of_hyperparameter_runs(hyp_seq_grid):
   scenario_dist = -2 * np.dot(hyp_seq_grid, hyp_seq_grid.T) + np.sum(hyp_seq_grid**2, axis=1) + np.sum(hyp_seq_grid**2, axis=1)[:, np.newaxis]
   scenario_rows = [0] # start with row 1
   for i in range(0,hyp_seq_grid.shape[0]):
      cur_dist = -2 * np.dot(hyp_seq_grid, hyp_seq_grid[scenario_rows].T) + np.sum(hyp_seq_grid[scenario_rows]**2, axis=1) + np.sum(hyp_seq_grid**2, axis=1)[:, np.newaxis]
      elem_rank_by_distance = rd(-(((cur_dist**2)).sum(axis=1))**(0.5)).argsort()
      pos_elem = [x for x in elem_rank_by_distance if x not in scenario_rows]
      if len(pos_elem)>0:
         scenario_rows.append(pos_elem[0])
   return scenario_rows

# internal function for cross-validation: make validation sets and test sets
def make_cv_weights(n_obs, fHoldoutProportion = 0.5, kfold = 3, seed = 1000):
   # trainging set and test set
   ix_train, ix_test = train_test_split([i for i in range(0,n_obs)], test_size = fHoldoutProportion, random_state = seed)
   # divide the training data into cross-validation sets
   cv_splitter = KFold(n_splits = kfold)
   cv_sets = []
   for ix_insample, ix_validation in cv_splitter.split(ix_train):
      cv_sets.append([ix_insample, ix_validation])
   return ix_train, ix_test, cv_sets

# internal function for cross-validation: propose hyperparameters, by two methods:
# ... i) sequential (just loops through combinations)
# ... ii) thompson sampling / multi-arm bandit reinforcement learner (ridge regression & decision trees to try to predict what might be best hyperparameters)
def proposal_hyperparam(run_number, # iteration number for the algorithm
                        cv_results, # current results of the cv-loss (panda frame)
                        hyp_pd, # output from "make_hyperparameters_combos" function
                        hyp_optimal, # output from "make_hyperparameters_combos" function
                        toggle_learner = 10, # when to switch to mult-arm bandit learning
                        ridge_alpha = 7, # ridge regression learner: shrinkage parameter
                        max_depth = 2, # tree-learner maximum tree depth
                        multinomial_prior = 1): # diffuse prior on the model space for the multi-arm bandit 
   n_models = len(hyp_optimal)
   if (run_number < toggle_learner) | (run_number > (hyp_pd.shape[0]-1)):
      # just sequentially loop through hyperparameter combos
      print("next hyperparameter: estimated by maximum parameter contrast")
      return_hypIx = hyp_optimal[run_number]
      return_hyperparameters = hyp_pd.iloc[return_hypIx]
   else:
      print("next hyperparameter: Thompson sampling of hyperparameters based on a multi-arm bandit learner") 
        # find which hyperparam/combos are already completed (used for estimation)
      which_done = np.where(cv_results['best_epoch'].values >0)
      # find which hyperparam/combos are not yet run (used for predicting their loss)
      which_notdone = np.where(cv_results['best_epoch'].values == 0)
      # train a ridge regression 
      learner1 = Ridge(alpha=ridge_alpha, copy_X = True,normalize=True)
      # train a decision tree
      learner2 = DecisionTreeRegressor(max_depth = max_depth)
      # multi-arm bandit: use leave-one-out estimation to get frequency counts that each combo is the best
      counts_best_loss = np.zeros(len(which_notdone[0])) + multinomial_prior # notice shrinkage parameter multinomial_prior=1
      # loop through each completed combo and drop it (leave-one-out subsampling)
      for j in range(0,len(which_done[0])):
         # drop the j'th combo (leave-one-out)
         loo = which_done[0][np.arange(len(which_done[0]))!=j]
         # fit learners
         lr1 = learner1.fit(X = hyp_pd.values[loo], y = cv_results['cv_loss'].values[loo]) 
         lr2 = learner2.fit(X = hyp_pd.values[loo], y = cv_results['cv_loss'].values[loo]) 
         # predict which will be the lowest loss         
         pred_loss1 = lr1.predict(hyp_pd.values[which_notdone]) # expected loss from the learner
         pred_loss2 = lr2.predict(hyp_pd.values[which_notdone]) # expected loss from the learner         
         # increment number of time's each hypparam/combo is predicted to be the best
         counts_best_loss += 0.5*(pred_loss1 == min(pred_loss1))
         counts_best_loss += 0.5*(pred_loss2 == min(pred_loss2))
      # convert selection frequency of each combo into a probability (Dirichlet)
      thompson_sampling_prob = np.random.dirichlet(counts_best_loss)
      # Thompson sampler: random sample from the probabilities (Multinomial)
      return_hypIx = np.random.choice(which_notdone[0],p=thompson_sampling_prob)
      # get the index of the hyperparameters for the best expectd loss
      # hyperparameters for the best expectd loss
      return_hyperparameters = hyp_pd.iloc[return_hypIx]
   return return_hypIx, return_hyperparameters 



## Functions: K-fold Cross-Validation
The following functions support the 3-fold cross validation. The are fairly self-explanatory. The main functions are:
+ `make_cv_weights` : splits the data into two indices for training & testing. Among the training indices (`ix_train`), it further divides the data into mutually exclusive k-fold cross-validation sets (`cv_sets`). You tune the hyperparameters with the subsets in `cv_sets`; you train a final (tuned) model on the `ix_train`, and you validate the final model on the hold-out set `ix_test`.
+ `run_model` : runs an LSTM model using the `keras` API; 
+ `run_cv_model`: calls `run_model` for one CV-iteration; ONLY returns the validation loss. 

In [3]:
# internal function for cross-validation: make validation sets and test sets
def make_cv_weights(n_obs, fHoldoutProportion = 0.5, kfold = 3, seed = 1000):
   # trainging set and test set
   ix_train, ix_test = train_test_split([i for i in range(0,n_obs)], test_size = fHoldoutProportion, random_state = seed)
   # divide the training data into cross-validation sets
   cv_splitter = KFold(n_splits = kfold)
   cv_sets = []
   for ix_insample, ix_validation in cv_splitter.split(ix_train):
      cv_sets.append([ix_insample, ix_validation])
   return ix_train, ix_test, cv_sets

# function: LSTM model, using Keras functional API
# use the Keras API to build the model
# Main LSTM model. Two inputs: X=word features (tokenized); X_labels=matrix of categories [1,2,3,...,nLabels]
# ... the output is Y: one-hot-coding of labels for each row of X,X_labels.
# Notice the `RepeatVector` which merges the output from the LSTM with the output from the Category embedding
def run_model(X, # tokenized text
              X_labels, # categories vector [1,2,3,...]
              Y, # labels (one-hot-coding)
              W, # sample weights
              X_val, # out-of-sample validation: X
              X_labels_val, # # out-of-sample validation: X_labels
              Y_val, # # out-of-sample validation: Y
              max_tokens, # number of words/tokens used to tokenize sentences
              n_epoch = 30, # hyperparameter
              batch_size = 64, # hyperparameter
              dim_embed_lstm= 200, # word embedding dimension
              dim_out_lstm =100, # dimension of LSTM output
              fDropout_RNN=0.1, # regularization: recurrent_dropout for LSTM
              fDropout=0.33, # regularization: input dropout for LSTM
              dim_embed_categ=5, # embedding dimension of the categories
              dim_hidden_nodes_final =100): # number of hidden nodes in final Dense layer
   nLabels = X_labels.shape[1]
   # main input: the word features (tokenized) 
   lstm_input_layer = Input(shape=(X.shape[1],), dtype='int32', name='lstm_input',) # lstm input
   # word embedding (like word2vec)
   lstm_embed_layer = Embedding(input_dim=max_tokens, output_dim=dim_embed_lstm, input_length=X.shape[1])(lstm_input_layer)
   lstm_output = LSTM(dim_out_lstm, dropout = fDropout, recurrent_dropout = fDropout_RNN)(lstm_embed_layer)
   # reshape the LSTM output to concatenate with the category embedding
   lstm_reshape = RepeatVector(nLabels)(lstm_output)  
   label3_input_layer = Input(shape=(X_labels.shape[1],), dtype='int32', name='label3_input') # input the topics
   label3_embed_layer = Embedding(input_dim=nLabels, output_dim = dim_embed_categ, input_length=X_labels.shape[1])(label3_input_layer) # topic embedding: should have dim: None,7,embed_dim
   # merge the LSTM output with the category embedding
   x = concatenate([lstm_reshape,label3_embed_layer],axis=2) # 
   # final hidden layer
   hidden_layer = Dense(dim_hidden_nodes_final, activation='relu')(x)
   final_layer = Dense(1, activation='sigmoid')(hidden_layer) # main output for categories
   # reshape the output so that it is multinomial in [n_obs, n_categories]
   final_layer2 = Flatten()(final_layer) # dimension: [n_obs, n_categories]
   main_output = Dense(Y.shape[1], activation='softmax',name = 'main_output')(final_layer2) # main output for categories 
   model = Model(inputs=[lstm_input_layer, label3_input_layer], outputs=[main_output])
   model.compile(loss = "categorical_crossentropy", optimizer='adam') 
   history = model.fit({'lstm_input': X, 'label3_input': X_labels }, {'main_output': Y}, epochs = n_epoch, batch_size=batch_size, verbose = 0, sample_weight = W, validation_data=([X_val, X_labels_val], Y_val))
   return model, history

# main function: run individual cv-run; ONLY returns the validation loss (deletes model)
def run_cv_model(cv_insample_indices,cv_validation_indices, Y, cText, X_labels, vWeights, hyperparam, n_epoch = 7, fDropout_RNN = 0.1, batch_size = 64):
   n_obs = Y.shape[0] # number of observations/rows
   # get hyperparameters
   iMaxTokens = int(hyperparam['max_tokens']) # maximum number of words in corpus for embedding
   iDimEmbedLSTM = int(hyperparam['DimEmbedLSTM']) # embedding dimensions for words
   iDimOutLSTM = int(hyperparam['DimOutLSTM']) # LSTM output dimension
   fDropout = hyperparam['Dropout'] # LSTM input dropout regularization
   iDimEmbedCategory = int(hyperparam['DimEmbedCategory'])
   iDim_hidden_nodes_final = (np.linspace(iDimOutLSTM,Y.shape[1]).round().astype(int))[1] # number of    
   # tokenize the text
   tokenizer = Tokenizer(num_words=iMaxTokens, split=' ')
   tokenizer.fit_on_texts("STARTCODON " + cText[0])
   # text data: insample and validation
   X = pad_sequences(tokenizer.texts_to_sequences(("STARTCODON " + cText[0]).values)) # tokenize and pad with zeros
   X_insample = X[cv_insample_indices] 
   X_val = X[cv_validation_indices]
   # label data (N-hot-coding): insample and validation
   Y_insample = Y[cv_insample_indices]
   Y_val = Y[cv_validation_indices]
   # labels for category embedding
   X_labels_insample = X_labels[cv_insample_indices]
   X_labels_val = X_labels[cv_validation_indices]
   # training weights
   W_insample = vWeights[cv_insample_indices]
   # fit the model
   model, history = run_model(X_insample, X_labels_insample, Y_insample, W_insample, 
                              X_val,X_labels_val,Y_val, 
                              iMaxTokens, 
                              n_epoch, 
                              batch_size, 
                              iDimEmbedLSTM, 
                              iDimOutLSTM, 
                              fDropout_RNN, 
                              fDropout, 
                              iDimEmbedCategory, 
                              iDim_hidden_nodes_final)
   val_loss = history.history['val_loss']
   del model
   del history
   return val_loss

# internal function for cross-validation: get the best epoch over k-fold validation
def get_best_epoch(cv_losses):
      expected_loss_per_epoch = (np.array(cv_losses).T).mean(axis=1)
      best_epoch = expected_loss_per_epoch.argmin() + 1
      return best_epoch, expected_loss_per_epoch[best_epoch-1]

# internal function for cross-validation: get best hyperparameter combinations from CV-loss results
def get_best_hyperparameters(cv_results, hyperparameter_grid):
   which_done = np.where(cv_results['best_epoch'].values!=0)
   which_best = which_done[0][cv_results['cv_loss'].values[which_done[0]].argmin()]
   best_epoch = cv_results['best_epoch'][which_best]
   return hyperparameter_grid.iloc[which_best], best_epoch 


## Script: Part 1: import and process data

+ NLP the text data (`cText`). 
+ Convert the categories/labels into one-hot-coding (`Y`)
+ Make the category embedding (`X_labels3`).
+ Make sample weights (`vWeights`) for the keras `sample_weight` argument.

In [4]:
# import and clean the data.
# Returns 'cText' (the NLP-preprocessed data) and 'cats' (the dictionary of categories/labels for classification')
cText, Y, X_labels3, cats = import_and_clean_data(filename = "complaints-2018-09-30_17_53.csv.tar.xz", # filename to import the data,
    #filename = "complaints-2018-09-30_17_53.csv", # filename to import the data,     
                                    col_label = "Label3", # label to work with for classification/learning
                                    data_dir = "data/", # directory
                                    tmp_dir = "/tmp/", # if file is a .tar.xz, where to temporarily extract the data (Windows users need specify differently than /tmp/
                                    rare_categories_cutoff = 10, # threshold for categories to be included in the training set
                                    word_clip = 300) # max number of words in text to accept (only first 300 words are retained

n_obs = Y.shape[0]
nLabels3 = X_labels3.shape[1]
print("number of observations:" + str(n_obs))
print("number of categories:" + str(nLabels3))
      
# sample weights to correct for unbalanced categories / labels
vWeights = get_class_weights(Y,5) # weights


decompressing complaints-2018-09-30_17_53.csv.tar.xz into /tmp/
tar xf data/complaints-2018-09-30_17_53.csv.tar.xz -C /tmp/
imported 191828 rows of data
performing NLP pre-processing on column Consumer complaint narrative (remove stop words, stemming,...). This may take a while...
Done NLP pre-processing
returning cleaned text data 'cText' for use in tokenization; 'Y' as an one-hot-coding matrix of categories; and 'cats' a dictionary matches columns in Y to english category names
deleting temporary file complaints-2018-09-30_17_53.csv.tar.xz
rm /tmp/complaints-2018-09-30_17_53.csv
done pre-processing
number of observations:191193
number of categories:425


## Script: Part 2: define the hyperparameters and set-up cross-validation
### validation sets
+ `kfold` : used for k-fold cross-validation; I use 3-fold validation
+ `fHoldoutProportion` : used to define the proportion of data not-used for training, but for validating the final model.

### hyperparameter space
+ `max_tokens`: the size of the corpus used in the word-tokenization (which feeds into the word-embedding).
+ `DimEmbedLSTM`:the embedding dimension of the word-embedding (like word2vec) 
+ `DimOutLSTM` : the dimensionality of the Long-Short-Term-memory outputs
+ `Dropout` regularization parameter: dropout rate in the LSTM
+ `DimEmbedCategory`: the embedding dimension of the category-embedding

In [6]:
kfold = 3 # k-fold cross-validation
fHoldoutProportion = 0.8 # amount of data to leave aside for (final) hold-out validation

# hyperparmeters' and their values
hyper_parameters = {'max_tokens': [1500,2000,3000], # number of wor tokens
                    'DimEmbedLSTM':[97,194,291], # word embedding dimension
                    'DimOutLSTM':[100,150,300], # LSTM output dimension
                    'Dropout':[0.20,0.33,0.5], # dropout proportion for LSTM
                    'DimEmbedCategory':[2,4,8,12,16,20]} # category embedding

# hyperparameters: make all possible combinations of 
hyp_grid, ix_hyp_grid, cv_results = make_hyperparameters_combos(hyper_parameters)
optimal_hyp_order = optimal_order_of_hyperparameter_runs(ix_hyp_grid)

# training set, testing sets, and cross-validation sets: indicies to divide the data
ix_train,ix_test,cv_sets = make_cv_weights(n_obs, fHoldoutProportion = fHoldoutProportion, kfold = kfold, seed = 1000)

n_epoch = 25 # number of epochs
batch_size = 200 # batchsize
pause_seconds = 120 # pause between CV-iterations

# big loop to run through hyperparameters
max_hyperparameter_runs = min(20, len(optimal_hyp_order)) # maximum number of iterations


## Script: Part 3: Explore the hyperparameter space (cross-validation)
This is the main part of the exercise, consisting of two massive loops. The outer loop iterates through different hyperparameter values, and the inner loop iterates through the K-fold cross-validation

In [7]:
i = 0 # iterator, outer loop
print("running k-fold hyperparameter estimation algorithm for " + str(max_hyperparameter_runs) + "iterations")
while i < max_hyperparameter_runs:
   print("hyperparameter tuning, iteration: " + str(i))
   # proposal new hyperparameters from the space of combinations
   ixHyper, current_hyper_param = proposal_hyperparam(i, cv_results, hyp_pd = hyp_grid, hyp_optimal = optimal_hyp_order, toggle_learner = 10, ridge_alpha = 7)
   print(current_hyper_param.values)
   # do cross-validation
   k_loss = [] # validation loss results
   # inner loop, cross-validation
   for icv, lcv_set in enumerate(cv_sets):
      k_loss.append(run_cv_model(lcv_set[0],lcv_set[1], Y, cText, X_labels3, vWeights, current_hyper_param, n_epoch = n_epoch, fDropout_RNN = 0.1, batch_size = batch_size))
   # return best mean(validation_loss) at which epoch 
   best_epoch, k_expected_loss = get_best_epoch(k_loss) # get best epoch and expected loss
   cv_results['best_epoch'].iloc[ixHyper] = best_epoch
   cv_results['cv_loss'].iloc[ixHyper] = k_expected_loss
   print("done iteration "+ str(i)+ ". Sleeping for " + str(pause_seconds) +" seconds.")
   time.sleep(pause_seconds)
   # repeat
   i += 1

print("done k-fold hyperparameter estimation: hit " + str(max_hyperparameter_runs) + " iterations")


running k-fold hyperparameter estimation algorithm for 20iterations
hyperparameter tuning, iteration: 0
next hyperparameter: estimated by maximum parameter contrast
[1.5e+03 9.7e+01 1.0e+02 2.0e-01 2.0e+00]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         (None, 301)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 301, 97)      145500      lstm_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 100)          79200       embedding_1[0][0]                
__________________________________________________________________________________________________
l

done iteration 0. Sleeping for 120 seconds.
hyperparameter tuning, iteration: 1
next hyperparameter: estimated by maximum parameter contrast
[3.00e+03 2.91e+02 3.00e+02 5.00e-01 2.00e+01]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         (None, 301)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 301, 291)     873000      lstm_input[0][0]                 
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 300)          710400      embedding_7[0][0]                
__________________________________________________________________________________________________
label3_input (InputL

/home/rob/.local/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


done iteration 1. Sleeping for 120 seconds.
hyperparameter tuning, iteration: 2
next hyperparameter: estimated by maximum parameter contrast
[1.50e+03 1.94e+02 1.00e+02 2.00e-01 2.00e+00]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         (None, 301)          0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 301, 194)     291000      lstm_input[0][0]                 
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 100)          118000      embedding_13[0][0]               
__________________________________________________________________________________________________
label3_input (InputL

done iteration 2. Sleeping for 120 seconds.
hyperparameter tuning, iteration: 3
next hyperparameter: estimated by maximum parameter contrast
[3.0e+03 9.7e+01 3.0e+02 5.0e-01 2.0e+01]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         (None, 301)          0                                            
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 301, 97)      291000      lstm_input[0][0]                 
__________________________________________________________________________________________________
lstm_10 (LSTM)                  (None, 300)          477600      embedding_19[0][0]               
__________________________________________________________________________________________________
label3_input (InputLayer)

KeyboardInterrupt: 

# Script: Part 4: Get best hyperparameters (tuning)

In [ ]:
# process the CV results: get best hyperparameters
best_hyperparameters, n_epoch = get_best_hyperparameters(cv_results, hyp_grid)
cv_results = pd.concat([pd.DataFrame(hyp_grid),cv_results],axis=1)
cv_results = cv_results[cv_results['best_epoch']!=0]
cv_results.sort_values(by=['cv_loss'])
print(cv_results) # print the results of cross-validation

# Final best hyperparameters
iMaxTokens = int(best_hyperparameters['max_tokens'])
iDimEmbedLSTM = int(best_hyperparameters['DimEmbedLSTM'])
iDimOutLSTM = int(best_hyperparameters['DimOutLSTM'])
fDropout = best_hyperparameters['Dropout']
iDim_hidden_nodes_final = (np.linspace(iDimOutLSTM,Y.shape[1]).round().astype(int))[1] # number of


# Script: Part 5: Run the final 'tuned' model

In [ ]:
# fit final model
tokenizer = Tokenizer(num_words=iMaxTokens, split=' ')
tokenizer.fit_on_texts("STARTCODON " + cText[0])
# text data: insample and validation
X = pad_sequences(tokenizer.texts_to_sequences(("STARTCODON " + cText[0]).values)) # tokenize and pad with zeros
X_train = X[ix_train];
Y_train = Y[ix_train]
W_train = vWeights[ix_train]
# final model
lstm_input_layer = Input(shape=(X_train.shape[1],), dtype='int32', name='lstm_input',) # lstm input
lstm_embed_layer = Embedding(input_dim=iMaxTokens, output_dim=iDimEmbedLSTM, input_length=X_train.shape[1])(lstm_input_layer) # input_dim = vocab size,
lstm_output = LSTM(iDimOutLSTM, dropout = fDropout, recurrent_dropout = fDropout_RNN)(lstm_embed_layer) # the output has dimension (None, 12)
hidden_layer = Dense(iDim_hidden_nodes_final, activation='relu')(lstm_output)
main_output = Dense(Y_train.shape[1], activation='softmax', name='main_output')(hidden_layer) # main output for categories
model = Model(inputs=[lstm_input_layer], outputs=[main_output])
model.compile(loss = "categorical_crossentropy", optimizer='adam')
model.fit({'lstm_input': X_train}, {'main_output': Y_train}, epochs = n_epoch, batch_size=batch_size, verbose = 2, sample_weight = W_train)


# Script: Part 6: Test the final model
Using the holdout data (indices `ix_test`), we will estimate hold-out statistics, like the ROC/AUC

In [ ]:
# holdout data for final validation
X_test = X[ix_test]; 
Y_test = Y[ix_test]; 

# validation on test set
pwide =  model.predict(X_test) # predicted probability matrix
pvec = pwide.flatten() # vectorize the prediction matrix 
yvec = Y_test.flatten() # vectorize the one-hot-coding classes

# calcualte fpr, tpr, and AUC scores
global_fpr, global_tpr, threshold = roc_curve(yvec, pvec)
global_roc_auc = auc(global_fpr, global_tpr) # 
print("Global AUC score=%f" % (global_roc_auc))

# plot the ROCurve
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic (out-of-sample)')
plt.plot(global_fpr, global_tpr, 'purple', label = 'Global CV-AUC = %0.2f' % global_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
#plt.savefig('img/global_roc_default_model.png')
plt.show() # to plot in your own terminal
#Global AUC score=0.949081

# calculate AUC scores per category
roc_topic = [] # container for per-category AUC scores
Y_labels_test = Y_test.argmax(axis=1)
for i in range(0,Y.shape[1]):
    tmptopic_fpr, tmptopic_tpr, threshold = roc_curve(Y_test[:,i].flatten(), pwide[:,i].flatten())
    roc_topic.append(auc(tmptopic_fpr, tmptopic_tpr))

# average ROC/AUC score overall categories
print("Average AUC (holdout) score is %f" % (sum(roc_topic)/len(roc_topic)))

n, bins, patches = plt.hist(roc_topic, 20, facecolor='blue', alpha=0.5)
plt.xlabel('AUC scores (per category)')
plt.ylabel('Frequency')
plt.title(r'cv-AUC scores per category')
plt.show()